IGS RINEX Spec:  ftp://igs.org/pub/data/format/rinex210.txt

```matlab
function OputRformatHeaderv2(fileTag,fileNumList,fileLen) %,WeekNum,TOW)

% open or creat a rinex file to save the output of a GPS software receiver
%fid=fopen('log.txt','wt');
% fileTag -- the same part of the divided files
% fileNumList -- File Number List
% displayInterval -- 显示间隔 s <=120s(2 min) ; 
%pathName =sprintf('L:\\DataBackup\\SeptentrioData\\%s',fileTag);
%pathName = sprintf('G:\\test\\%s',fileTag);
addpath('G:\test\');
YMDHMS(1,1:3) = sscanf(fileTag,'%4f%2f%2f');
YMDHMS(1,4:6) = [0 0 0];
WeekNum = double(floor( (datenum(YMDHMS)-datenum([1980 1 6 0 0 0])) / 7));
TOW = round( (datenum(YMDHMS)-datenum([1980 1 6 0 0 0])-WeekNum*7)*3600*24 );
TOW = TOW+16.00;
tic;
interval = fileLen; 

%xlswrite(fileName_w,0, 1);
%WeekNum=1689;
%TOW=439092;
S4Dis=[];
sigmaDis=[];
startTime=datenum([1980 1 6 0 0 0]) + WeekNum * 7 + (TOW + fileLen * (fileNumList(1)-1))/3600/24;
    for fileNum = fileNumList
        fileStartTime = TOW +(fileNum-1)*fileLen-double(fileNum>1)*300;
        startTimeStr=datestr(fileStartTime/3600/24,'HHMMSS'); 
        if(fileNum==1)
            fileName=sprintf('%s_%s_7200_HT_%d.h5',fileTag,startTimeStr,fileNum); 
        else
            fileName=sprintf('%s_%s_7500_HT_%d.h5',fileTag,startTimeStr,fileNum); 
        end
        
        disp(fileName);
        if fileNum == 1;%fileNumList(1)
            cutTime = 0;
        else
            cutTime = 300; %
        end
        stopTime = startTime+(fileNumList(end)-fileNumList(1)+1)*interval/3600/24-15/3600/24;
        OputRformatHeaderv2_single(fileName,fileLen,fileStartTime+cutTime,stopTime,cutTime);        
        clear fileName;        
    end
    disp('========================================================================');
end
% ========== Get the date

function OputRformatHeaderv2_single(fileName,fileLen,startTime,stopTime,cutTime)
fileName_w = strrep(fileName,'.h5','.12o');
fid=fopen(fileName_w,'w+');
LOG.L1PSR=hdf5read(fileName, '/L1PSR');
LOG.L2PSR=hdf5read(fileName, '/L2PSR');
LOG.L1ADR=hdf5read(fileName, '/L1ADR');
%LOG.L1CNR=hdf5read(fileName, '/L1Doppler');
LOG.L1CNR=hdf5read(fileName,'/L2ADR');

GPSTime=hdf5read(fileName,'/GPSTime');

% Time needs to be in GPSSeconds since 6-Jan 1980, not week,tow.
ttime(1,:)=double(GPSTime(1,:));

LOG.GPSTime(1,:)=floor(ttime(1,1:length(ttime)-1)./(3600*168));
LOG.GPSTime(2,:)=ttime(1,1:length(ttime)-1)-LOG.GPSTime(1,:).*(3600*168);

clear GPSTime ttime;

%ActiveSatList0 = find(~isnan(LOG.L1PSR(1:32,1)));
dateNumber_ini = datenum([1980 1 6 0 0 0]);
OputRformatHeadv2(LOG.GPSTime,fid,'G');
%startTime = double(round(3600*24*7*mod((startTime-dateNumber_ini)/7,1)));
for k=1:1:(length(LOG.GPSTime(1,:)))
    WeekNumber = LOG.GPSTime(1,k);
    TOW = LOG.GPSTime(2,k);
    if TOW<startTime
        continue;
    end
    %ActiveSatList = ActiveSatList0(LOG.L1PSR(ActiveSatList0,k)~=0);
    %ActiveSatList = ActiveSatList(find(~isnan(LOG.L1PSR(ActiveSatList,k))));
    ActiveSatList = find(LOG.L1PSR(1:32,k)~=0);
    ActiveSatList = ActiveSatList(find(~isnan(LOG.L1PSR(ActiveSatList,k))));
    ActiveSatList = ActiveSatList(find(LOG.L1PSR(ActiveSatList,k)>0));
    %if k<468*2
    %   ActiveSatList=ActiveSatList(ActiveSatList~=27);
    %end
    dateNumber = dateNumber_ini + WeekNumber * 7 + TOW/3600/24;
    if WeekNumber == 0
        break;
    end
    %strTime = sprintf('%-4d%-4d%-12.3f',hour,min,sec);  
    %fprintf(fid,' %-14s%-20s%-3s%1d',strDate,strTime,'0');
    dispDateStr= datestr(dateNumber,'yyyy mm dd HH MM SS.FFF');
    dispDateStr = dispDateStr(3:end);    
    stateFlag=blanks(2);
    stateFlag(2)='0';        
    fprintf(fid,'\n %s0000 %s %2d',...
        dispDateStr,stateFlag,length(ActiveSatList));
        for j=1:length(ActiveSatList)
            i=ActiveSatList(j);
            if i<10
                Gstr = sprintf('G0%d',i);
            else
                Gstr = sprintf('G%2d',i);
            end
            fprintf(fid,'%3s',Gstr);
        end       
        
        for j=1:length(ActiveSatList)            
            
            P1 = sprintf('%10.3f',LOG.L1PSR(ActiveSatList(j),k));
            ADR1 = sprintf('%10.3f',LOG.L1ADR(ActiveSatList(j),k));
            P2 = sprintf('%10.3f',LOG.L2PSR(ActiveSatList(j),k));            
            POW1 = sprintf('%10.3f',LOG.L1CNR(ActiveSatList(j),k));   
            tmp=blanks(16*4);            
            tmp((14-length(P1)+1):14)=P1;            
            tmp((16+(14-length(ADR1)+1)):(16+14))=ADR1;          
            if ~isempty(strfind(P2,'NaN'))
                fprintf(fid,'\n%s',tmp);
            else
                tmp((16*2+(14-length(P2)+1)):(16*2+14))=P2;                 
                tmp((16*3+(14-length(POW1)+1)):(16*3+14))=POW1;
                fprintf(fid,'\n%s',tmp);
            end
            
        end
    
end% end of k=1:1:(length(LOG.L1PSR(1,:)))

fclose(fid);
end

function OputRformatHeadv2(GPSTime,fid, GNSSflag) 
endIndex = length(GPSTime(1,:));
startTime = datenum([1980 1 6 0 0 0]) + GPSTime(1,1)*7 + GPSTime(2,1) / 3600/24;
for endIndex=endIndex:-1:1
    if GPSTime(1,endIndex)~=0
        endTime   = datenum([1980 1 6 0 0 0]) + GPSTime(1,endIndex)*7 + GPSTime(2,endIndex) / 3600/24;
        break;
    end
end
switch GNSSflag    
    case 'glonass'
        systemflag = 'R';
        obsType = '      4    C1    L1    P2    L2';
    case 'galileo'
        systemflag = 'E';
        obsType = '      4    C1    L1    E5    E5a';
    otherwise % case 'gps'
        systemflag = 'G(GPS)';
        obsType = '      4    C1    L1    P2    L2';
end
tmp1=blanks(80);
tmp1(6:9)='2.10';
tmp1(21:36) = 'OBSERVATION DATA';
tmp1(51:(50+length(systemflag))) = systemflag;
str2='RINEX VERSION / TYPE';
tmp1(61:(60+length(str2))) = str2;
fprintf(fid,'%s\n',tmp1);

tmp1=blanks(80);
str2='# / TYPES OF OBSERV';
tmp1(1:length(obsType))=obsType;
tmp1(61:(60+length(str2)))=str2;
fprintf(fid,'%s\n',tmp1);

tmp1=blanks(80);
str1='0.02';
str2='INTERVAL';
tmp1(7:10)=str1;
tmp1(61:(60+length(str2)))=str2;
fprintf(fid,'%s\n',tmp1);

tmp1=blanks(80);
str1=strcat( datestr(startTime,'yyyy    mm    dd    HH    MM    SS.FFF'),'    GPS');
str2='TIME OF FIRST OBS';
tmp1(3:(2+length(str1)))=str1;
tmp1(61:(60+length(str2)))=str2;
fprintf(fid,'%s\n',tmp1);

tmp1=blanks(80);
str1=strcat( datestr(endTime,'yyyy    mm    dd    HH    MM    SS.FFF'),'    GPS');
str2='TIME OF LAST OBS';
tmp1(3:(2+length(str1)))=str1;
tmp1(61:(60+length(str2)))=str2;
fprintf(fid,'%s\n',tmp1);

tmp1=blanks(80);
str2='END OF HEADER';
tmp1(61:(60+length(str2)))=str2;
fprintf(fid,'%s\n',tmp1);

end
```